In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from collections import Counter
import warnings

In [ ]:
# Suppress specific warnings from scikit-learn for cleaner output
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")


In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/softwarica/machine-learning/air-quality-prediction-classification/compiled/kathmandu_pm25_class_2020_1_to_2025_4_dataset.csv')

In [ ]:
# Label Encode the target variable 'pm25_class'
le = LabelEncoder()
final_df['pm25_class'] = le.fit_transform(final_df['pm25_class'])

In [ ]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    """
    A custom transformer to perform complex feature engineering steps:
    - Clipping numerical features to specified bounds.
    - Transforming wind direction into sine/cosine components and binary flags.
    - Creating an 'is_windy' flag and handling rare 'condition' categories.
    - Generating lagged features for key variables including the target.
    - Dropping the 'date' column and handling NaNs.
    """
    def __init__(self, clipping_bounds=None, rare_condition_threshold=100, lag_features=None, lags=None):
        # Define default clipping bounds
        self.clipping_bounds = clipping_bounds if clipping_bounds is not None else {
            'temperature': {'lower': 1.5, 'upper': 35.4},
            'pressure': {'lower': 855, 'upper': 885},
            'dew_point': {'lower': -10, 'upper': 25},
            'humidity': {'lower': 0, 'upper': 100}
        }
        self.rare_condition_threshold = rare_condition_threshold
        # Define default features for lagging
        self.lag_features = lag_features if lag_features is not None else ['temperature', 'humidity', 'wind_speed', 'dew_point', 'pressure']
        self.lags = lags if lags is not None else [1, 2, 3]
        # Mapping for wind directions to degrees
        self.direction_map = {
            'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5,
            'E': 90, 'ESE': 112.5, 'SE': 135, 'SSE': 157.5,
            'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
            'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5,
            'CALM': np.nan, 'VAR': np.nan
        }
        self.rare_conditions_ = None # To store rare conditions learned during fit

    def fit(self, X, y=None):
        # Ensure X is a DataFrame for proper column operations
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        # Learn rare conditions from the 'condition' column
        if 'condition' in X.columns:
            temp_condition_base = X['condition'].str.replace(' / Windy', '', regex=False)
            condition_counts = temp_condition_base.value_counts()
            self.rare_conditions_ = condition_counts[condition_counts < self.rare_condition_threshold].index
        return self

    def transform(self, X):
        X_transformed = X.copy()

        # 1. Clipping numerical features according to predefined bounds
        for col, bounds in self.clipping_bounds.items():
            if col in X_transformed.columns:
                X_transformed[col] = X_transformed[col].clip(lower=bounds['lower'], upper=bounds['upper'])

        # 2. Wind Transformation: Convert wind direction to numerical features
        if 'wind' in X_transformed.columns:
            X_transformed['wind_deg'] = X_transformed['wind'].map(self.direction_map)
            X_transformed['is_calm'] = (X_transformed['wind'] == 'CALM').astype(int)
            X_transformed['is_var'] = (X_transformed['wind'] == 'VAR').astype(int)
            # Fill NaN degrees (from 'CALM'/'VAR') with 0 for trigonometric encoding
            X_transformed['wind_deg'] = X_transformed['wind_deg'].fillna(0)
            X_transformed['wind_sin'] = np.sin(np.deg2rad(X_transformed['wind_deg']))
            X_transformed['wind_cos'] = np.cos(np.deg2rad(X_transformed['wind_deg']))
            # Drop the original 'wind' column and the intermediate 'wind_deg'
            X_transformed.drop(columns=['wind', 'wind_deg'], inplace=True)

        # 3. Condition Transformation: Create 'is_windy' and handle rare categories
        if 'condition' in X_transformed.columns:
            X_transformed['is_windy'] = X_transformed['condition'].str.contains('/ Windy', na=False).astype(int)
            X_transformed['condition_base'] = X_transformed['condition'].str.replace(' / Windy', '', regex=False)
            # Replace rare conditions with 'Other' based on what was learned during fit
            if self.rare_conditions_ is not None:
                X_transformed['condition'] = X_transformed['condition_base'].replace(self.rare_conditions_, 'Other')
            else:
                # Fallback if fit was not called or no rare conditions were identified
                X_transformed['condition'] = X_transformed['condition_base']
            X_transformed.drop(columns=['condition_base'], inplace=True)

        # 4. Add lagged features: Requires 'date' column for sorting and 'pm25_class' for lagging.
        # This transformer is designed to be applied to the full dataset (including target)
        # before splitting into X and y for train/test.
        if 'date' in X_transformed.columns:
            X_transformed.sort_values('date', inplace=True) # Sort by date for correct lagging
            # Lag numerical features
            for feature in self.lag_features:
                if feature in X_transformed.columns:
                    for lag in self.lags:
                        X_transformed[f'{feature}_lag{lag}'] = X_transformed[feature].shift(lag)
            # Lag the PM2.5 class (target) as a feature
            if 'pm25_class' in X_transformed.columns:
                for lag in self.lags:
                    X_transformed[f'pm25_class_lag{lag}'] = X_transformed['pm25_class'].shift(lag)

            X_transformed.drop(columns=['date'], inplace=True) # Drop date column after lagging

        # Drop rows with NaNs created by lagging (first few rows will have NaNs)
        X_transformed.dropna(inplace=True)

        # Reset index to ensure clean DataFrame for subsequent steps in the pipeline
        return X_transformed.reset_index(drop=True)

In [ ]:
# --- Apply the Custom Feature Engineer and split X, y ---
# The FeatureEngineer needs to be fitted and transformed on the full dataset (final_df)
# before splitting into X and y, because it creates lagged features from the target.
df_engineered = FeatureEngineer().fit_transform(final_df.copy())


In [ ]:
# Separate features (X) and target (y) from the engineered DataFrame
X = df_engineered.drop(columns=['pm25_class'])
y = df_engineered['pm25_class']

In [ ]:
# --- Train Test Split ---
# Split the dataset into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
# Split the training+validation set into actual training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)


In [ ]:
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist()

In [ ]:
if 'condition' in numerical_cols:
    numerical_cols.remove('condition')
if 'condition' not in categorical_cols:
    categorical_cols.append('condition')

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ],
    remainder='passthrough'
)

In [ ]:
# Each pipeline consists of the preprocessor, SMOTE for oversampling, and a classifier.
pipeline_lr = Pipeline([
    ('preprocessor', preprocessor), # Applies scaling and one-hot encoding
    ('smote', SMOTE(random_state=42)), # Handles class imbalance on the training data
    ('classifier', LogisticRegression(random_state=42))  #max_iter=2000, solver='liblinear'))
])
pipeline_rf = Pipeline([
    ('preprocessor', preprocessor), # Applies scaling and one-hot encoding
    ('smote', SMOTE(random_state=42)), # Handles class imbalance on the training data
    ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
])

pipeline_xgb = Pipeline([
    ('preprocessor', preprocessor), # Applies scaling and one-hot encoding
    ('smote', SMOTE(random_state=42)), # Handles class imbalance on the training data
    ('classifier', xgb.XGBClassifier(random_state=42, n_jobs=-1, eval_metric='mlogloss', verbosity=0))
])

In [ ]:
# T specify the hyperparameters to tune for each classifier within the pipeline.
param_grids = {
     "Logistic Regression": {
        'classifier__C': [0.1, 1, 10],
    },
    "Random Forest": {
        'classifier__n_estimators': [100, 200], # Number of trees
        'classifier__max_depth': [10, 20],      # Maximum depth of each tree
    },
    "XGBoost": {
        'classifier__n_estimators': [100, 200], # Number of boosting rounds
        'classifier__max_depth': [3, 6],        # Maximum depth of each tree
        'classifier__learning_rate': [0.1, 0.2],# Step size shrinkage to prevent overfitting
    },
}

In [ ]:

# Train and Evaluate Pipelines using GridSearchCV
# Determine cross-validation folds based on the minimum class count in the training data
min_class_count = y_train.value_counts().min()
cv_folds = 2 if min_class_count < 5 else 5 # Use 2 folds if min class count is very low, else 5
cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

best_models = {}

# GridSearchCV for pipelines
print("Training Logistic Regression Pipeline...")
grid_lr = GridSearchCV(pipeline_lr, param_grids["Logistic Regression"], cv=cv, scoring='f1_macro', n_jobs=-1)
grid_lr.fit(X_train, y_train) # Fit on X_train; SMOTE and preprocessing are handled within the pipeline

print("Training Random Forest Pipeline...")
grid_rf = GridSearchCV(pipeline_rf, param_grids["Random Forest"], cv=cv, scoring='f1_macro', n_jobs=-1)
grid_rf.fit(X_train, y_train) # Fit on X_train; SMOTE and preprocessing are handled within the pipeline

print("Training XGBoost Pipeline...")
grid_xgb = GridSearchCV(pipeline_xgb, param_grids["XGBoost"], cv=cv, scoring='f1_macro', n_jobs=-1)
grid_xgb.fit(X_train, y_train) # Fit on X_train; SMOTE and preprocessing are handled within the pipeline

# Store the best models from each pipeline
best_models["Logistic Regression"] = grid_lr.best_estimator_
best_models["Random Forest"] = grid_rf.best_estimator_
best_models["XGBoost"] = grid_xgb.best_estimator_



Training Logistic Regression Pipeline...
Training Random Forest Pipeline...
Training XGBoost Pipeline...


In [ ]:
print("\nBest Parameters for Logistic Regression:", grid_lr.best_params_)
print("Logistic Regression CV F1 Score:", grid_lr.best_score_)

print("\nLogistic Regression Validation Report:")
val_preds_lr = grid_lr.predict(X_val)
print(classification_report(y_val, val_preds_lr, zero_division=0))

val_accuracy_lr = accuracy_score(y_val, val_preds_lr)
print(f"Logistic Regression Validation Accuracy: {val_accuracy_lr:.4f}")


Best Parameters for Logistic Regression: {'classifier__C': 0.1}
Logistic Regression CV F1 Score: 0.5613921462375991

Logistic Regression Validation Report:
              precision    recall  f1-score   support

           0       0.21      0.57      0.31         7
           1       0.54      0.56      0.55        34
           2       0.40      0.54      0.46        41
           3       0.87      0.65      0.74       140
           4       0.66      0.76      0.71        51

    accuracy                           0.64       273
   macro avg       0.54      0.62      0.55       273
weighted avg       0.70      0.64      0.66       273

Logistic Regression Validation Accuracy: 0.6410


In [ ]:
print("\nBest Parameters for Random Forest:", grid_rf.best_params_)
print("Random Forest CV F1 Score:", grid_rf.best_score_)

print("\nRandom Forest Validation Report:")
val_preds_rf = grid_rf.predict(X_val)
print(classification_report(y_val, val_preds_rf, zero_division=0))

val_accuracy_rf = accuracy_score(y_val, val_preds_rf)
print(f"Random Forest Validation Accuracy: {val_accuracy_rf:.4f}")


Best Parameters for Random Forest: {'classifier__max_depth': 10, 'classifier__n_estimators': 200}
Random Forest CV F1 Score: 0.6519071819790677

Random Forest Validation Report:
              precision    recall  f1-score   support

           0       0.62      0.71      0.67         7
           1       0.73      0.65      0.69        34
           2       0.55      0.68      0.61        41
           3       0.84      0.77      0.81       140
           4       0.64      0.71      0.67        51

    accuracy                           0.73       273
   macro avg       0.68      0.70      0.69       273
weighted avg       0.74      0.73      0.73       273

Random Forest Validation Accuracy: 0.7289


In [ ]:

print("\nBest Parameters for XGBoost:", grid_xgb.best_params_)
print("XGBoost CV F1 Score:", grid_xgb.best_score_)

# Evaluate XGBoost
print("\nXGBoost Validation Report:")
val_preds_xgb = grid_xgb.predict(X_val)
print(classification_report(y_val, val_preds_xgb, zero_division=0))

val_accuracy_xgb = accuracy_score(y_val, val_preds_xgb)
print(f"XGBoost Validation Accuracy: {val_accuracy_xgb:.4f}")


Best Parameters for XGBoost: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 200}
XGBoost CV F1 Score: 0.6573208968693166

XGBoost Validation Report:
              precision    recall  f1-score   support

           0       0.43      0.43      0.43         7
           1       0.64      0.74      0.68        34
           2       0.57      0.56      0.57        41
           3       0.84      0.79      0.81       140
           4       0.66      0.73      0.69        51

    accuracy                           0.73       273
   macro avg       0.63      0.65      0.64       273
weighted avg       0.73      0.73      0.73       273

XGBoost Validation Accuracy: 0.7253


In [ ]:
# --- Final Test Evaluation ---
# Evaluate the best trained models on the unseen test set
print("\nFinal Test Evaluation:")
for name, model in best_models.items():
    test_preds = model.predict(X_test)
    print(f"\n{name} Test Report:")
    print(classification_report(y_test, test_preds, zero_division=0))


Final Test Evaluation:

Random Forest Test Report:
              precision    recall  f1-score   support

           0       0.67      0.57      0.62         7
           1       0.63      0.65      0.64        34
           2       0.69      0.61      0.65        41
           3       0.86      0.85      0.85       139
           4       0.69      0.77      0.73        52

    accuracy                           0.77       273
   macro avg       0.71      0.69      0.70       273
weighted avg       0.77      0.77      0.77       273


XGBoost Test Report:
              precision    recall  f1-score   support

           0       0.83      0.71      0.77         7
           1       0.62      0.59      0.61        34
           2       0.64      0.56      0.60        41
           3       0.85      0.89      0.87       139
           4       0.74      0.75      0.74        52

    accuracy                           0.77       273
   macro avg       0.74      0.70      0.72       273
wei

In [ ]:
import joblib

# Assuming 'best_models' dictionary contains your trained pipelines
# best_models["Random Forest"] and best_models["XGBoost"]
joblib.dump(best_models["Random Forest"], 'random_forest_pipeline.pkl')
joblib.dump(best_models["XGBoost"], 'xgboost_pipeline.pkl')

# Also save the LabelEncoder and the X_train_columns for consistent preprocessing
joblib.dump(le, 'label_encoder.pkl')
joblib.dump(X_train.columns.tolist(), 'X_train_columns.pkl')

# Save the accuracy scores
joblib.dump(val_accuracy_rf, 'rf_validation_accuracy.pkl')
joblib.dump(val_accuracy_xgb, 'xgb_validation_accuracy.pkl')

print("Models, LabelEncoder, X_train_columns, and Validation Accuracies saved successfully!")